<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/Do_An_LinhTinh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!sudo apt install build-essential cmake pkg-config unzip yasm git checkinstall

!sudo apt install libjpeg-dev libpng-dev libtiff-dev
!sudo apt install libavcodec-dev libavformat-dev libswscale-dev libavresample-dev
!sudo apt install libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev
!sudo apt install libxvidcore-dev x264 libx264-dev libfaac-dev libmp3lame-dev libtheora-dev 
!sudo apt install libfaac-dev libmp3lame-dev libvorbis-dev

!sudo apt-get install libatlas-base-dev gfortran
!sudo apt-get install libprotobuf-dev protobuf-compiler
!sudo apt-get install libgoogle-glog-dev libgflags-dev
!sudo apt-get install libgphoto2-dev libeigen3-dev libhdf5-dev doxygen



Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
unzip is already the newest version (6.0-21ubuntu1).
checkinstall is already the newest version (1.6.2-4ubuntu2).
yasm is already the newest version (1.3.0-2build1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
libpng-dev is already the newest version (1.6.34-1ubuntu0.18.04.2).
libtiff-dev is already the newest

In [7]:
%cd ~
%cd opencv-4.1.2
!cmake -D CMAKE_BUILD_TYPE=RELEASE \ -D CMAKE_INSTALL_PREFIX=/usr/local \
-D INSTALL_PYTHON_EXAMPLES=ON \
-D INSTALL_C_EXAMPLES=OFF \
-D WITH_TBB=ON \
-D WITH_CUDA=ON \
-D BUILD_opencv_cudacodec=OFF \
-D ENABLE_FAST_MATH=1 \
-D CUDA_FAST_MATH=1 \
-D WITH_CUBLAS=1 \
-D WITH_V4L=ON \
-D WITH_QT=OFF \
-D WITH_OPENGL=ON \
-D WITH_GSTREAMER=ON \
-D OPENCV_GENERATE_PKGCONFIG=ON \
-D OPENCV_PC_FILE_NAME=opencv.pc \
-D OPENCV_ENABLE_NONFREE=ON \
-D OPENCV_PYTHON3_INSTALL_PATH=~/<virtualenv>/lib/python3.5/site-packages \
-D OPENCV_EXTRA_MODULES_PATH=~/opencv_contrib-4.2.0/modules \
-D PYTHON_EXECUTABLE=~/<virtualenvs>/bin/python \
-D BUILD_EXAMPLES=ON
-D CUDA_ARCH_BIN=7.5
-D BUILD_SHARED_LIBS=OFF ..

'4.1.2'

In [6]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Tiền xử lý dữ liệu và trích xuất đặc trưng 

**hàm xử lý dữ liệu và trích xuất đặc trưng bằng HOG**

In [45]:
def CvCanny(matrix):
  v = np.median(matrix)
  sigma = 0.33
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  canny = cv2.Canny(matrix, lower, upper, True)
  return canny

In [46]:
def apply_roi(img, roi):
  #thresh, roi = cv2.threshold(roi, thresh=128, maxval=1, type=cv2.THRESH_BINARY)
  new_img = img * roi
  return new_img

In [47]:
def extract_features(path, label, data, labels):
  videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
  i = 150;
  for videoPath in videoPaths:
    cap = cv2.VideoCapture(videoPath)
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (160, 120))
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    if i <= 0:
        break
    i -= 1
    s = 1
    roi = []
    while(cap.isOpened()):
        ret, frame2 = cap.read()

        if not ret:
          break;
        if s % 6 == 0:
          frame2 = cv2.resize(frame2, (160, 120))
          #Trích xuất dòng quang từ 2 frame gần nhau
          next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
          optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
          flow = optical_flow.calc(prvs, next, None)
          if s == 3:
            flowx = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flowy = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
          mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
          hsv[...,0] = ang*180/np.pi/2
          nom = np.zeros((160,120))
          hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
          bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
          gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

          #Tăng độ nét cho ảnh dòng quang
          '''kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
          sharpen = cv2.filter2D(hsv, -1, kernel)
          canny1 = CvCanny(sharpen)'''
          canny2 = CvCanny(gray)
          #gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
          #cv2_imshow(gray)
          
          '''gray1 = cv2.filter2D(gray, -1, kernel)[...,::-1]
          gray = cv2.ximgproc.guidedFilter(gray, gray1, radius=2, eps=0.004, dDepth=-1)[...,::-1]
          canny3 = CvCanny(gray)
          cv2_imshow(gray)'''
          

          kernel = np.ones((5,5),np.uint8)
          closing = cv2.morphologyEx(canny2, cv2.MORPH_CLOSE, kernel)
          closing_inv = cv2.bitwise_not(closing)
          
          h, w = closing.shape[:2]
          mask = np.zeros((h+2, w+2), np.uint8)
          
          '''im_floodfill = closing.copy()
          cv2.floodFill(im_floodfill, mask, (0,0), 255);
          im_floodfill_inv = cv2.bitwise_not(im_floodfill)
          floodfill = closing | im_floodfill'''

          contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

          for cnt in contour:
            cv2.drawContours(closing,[cnt],0,255,-1)

          opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
          #opening = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
          '''cv2_imshow(canny1)
          cv2_imshow(canny2)
          cv2_imshow(canny3)'''

          after_roi = apply_roi(next, opening)

          # trích xuất feature bằng HOG
          winSize = (32,32)
          blockSize = (16,16)
          blockStride = (8,8)
          cellSize = (8,8)
          nbins = 9
          hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
          image_hog = hog.compute(after_roi)

          data.append(image_hog)
          labels.append(label)


          prvs = next.copy()


        s += 1
    print(videoPath + "---done--")



**Khởi tạo các biến chứa nhãn và dữ liệu**

In [48]:
labels = []
data = []

**Tiến hành trích xuất đặc trưng lưu vào các biến data và labels**

In [49]:
path = "/content/drive/My Drive/Machine Lol/Data/cobaoluc"
extract_features(path, 0, data, labels)
path = "/content/drive/My Drive/Machine Lol/Data/khongbaoluc"
extract_features(path, 1, data, labels)

/content/drive/My Drive/Machine Lol/Data/cobaoluc/0.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/1.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/45.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/75.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/95.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/98.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/7.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/68.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/71.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/88.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/77.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/8.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/99.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/80.mp4---done--
/content/drive/My Drive/Machine Lol/Data/cobaoluc/96.mp4---done--
/content/drive

In [50]:
print(len(data))
print(len(labels))

4021
4021


**Xử lý các feature để có thêm đưa vào model train được**

In [51]:
print(data[1][1:5])

[[0.00720486]
 [0.0181789 ]
 [0.21510355]
 [0.38910678]]


In [ ]:
'''conv_data = []
for hog_image in data:
  temp = []
  for scalar in hog_image:
    temp.append(scalar.item())
  
  conv_data.append(temp)'''
conv_data1 = [[j.item() for j in i] for i in data]

print(conv_data1[1][1:5])

**Chia dữ liệu train và test**

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(conv_data1, labels,
	test_size=0.20, random_state=42)

# Train model

**Model svm**

In [ ]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(trainX, trainY))
print(lsvc.score(testX, testY))

**Model RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

rfc.fit(trainX, trainY)

print(rfc.score(trainX, trainY))
print(rfc.score(testX, testY))

**Model GaussianNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(trainX, trainY)

print(gnb.score(trainX, trainY))
print(gnb.score(testX, testY))


**Model LogisticRegression**

In [ ]:

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(trainX, trainY)

print(lr.score(trainX, trainY))
print(lr.score(testX, testY))
